In [ ]:
import os
import glob
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout, Input, concatenate, LSTM, Permute, Attention, Lambda, Dot, Activation, Concatenate, Layer, Flatten, Conv1D, MaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, Embedding

# Load data
daily_data = np.load('sea-ice-prediction-main/sea-ice-prediction-main/climate-change-ai-workshop/data/dailyt30_features.npy', allow_pickle=True)
monthly_data = np.load('sea-ice-prediction-main/sea-ice-prediction-main/climate-change-ai-workshop/data/monthly_features.npy', allow_pickle=True)
daily_target = np.load('sea-ice-prediction-main/sea-ice-prediction-main/climate-change-ai-workshop/data/dailyt30_target.npy', allow_pickle=True)
monthly_target = np.load('sea-ice-prediction-main/sea-ice-prediction-main/climate-change-ai-workshop/data/monthly_target.npy', allow_pickle=True)


# Data preprocessing
daily_data = daily_data[:-7, :, :]
monthly_data = monthly_data[:-7, :, :]
monthly_target = monthly_target[:-7]

data = np.concatenate((daily_data, monthly_data), axis=1)
lag = 1
data = data[:-lag, :, :]
monthly_target = monthly_target[lag:]

LEN_DATA = len(data)
NUM_TRAIN = LEN_DATA - (12 * 20)
NUM_VALID = LEN_DATA - NUM_TRAIN

x_train = data[0:NUM_TRAIN]
x_valid = data[NUM_TRAIN:]

y_train = monthly_target[:NUM_TRAIN]
y_valid = monthly_target[NUM_TRAIN:]

# Normalize features
scaler_f = StandardScaler()
x_train = scaler_f.fit_transform(x_train.reshape(-1, x_train.shape[2]))
x_valid = scaler_f.transform(x_valid.reshape(-1, x_valid.shape[2]))

scaler_l = StandardScaler()
y_train = scaler_l.fit_transform(y_train.reshape(-1, 1))
y_valid = scaler_l.transform(y_valid.reshape(-1, 1))

# Reshape features
def reshape_features(dataset, timesteps=1):
    return dataset.reshape((int(dataset.shape[0] / timesteps)), timesteps, dataset.shape[1])

timesteps = 31
x_train = reshape_features(x_train, timesteps)
x_valid = reshape_features(x_valid, timesteps)

# Split the reshaped features into two separate inputs for the model
x_train1 = x_train[:, :30, :]
x_train2 = x_train[:, 30:31, :]

x_valid1 = x_valid[:, :30, :]
x_valid2 = x_valid[:, 30:31, :]


import tensorflow as tf
from tensorflow.keras.layers import (Input, Dense, Dropout, LSTM, Conv1D,
                                     MultiHeadAttention, LayerNormalization,
                                     concatenate, Permute, Add, GlobalAveragePooling1D)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def build_advanced_model():
    # Input layers
    model1_input = Input(shape=(30, x_train1.shape[-1]))
    model2_input = Input(shape=(1, x_train2.shape[-1]))

    # Model 1
    model1 = Permute((2, 1))(model1_input)
    model1 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(model1)
    model1 = MultiHeadAttention(num_heads=8, key_dim=64)(model1, model1)
    model1 = Dropout(0.4)(model1)
    model1 = LSTM(256, return_sequences=True)(model1)
    model1 = Dropout(0.4)(model1)
    model1 = LSTM(128, return_sequences=True)(model1)
    model1 = Dropout(0.4)(model1)
    model1 = GlobalAveragePooling1D()(model1)
    model1_output = Dense(64, activation='relu')(model1)
    model1_output = Dense(1)(model1_output)

    # Model 2
    model2 = Permute((2, 1))(model2_input)
    model2 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(model2)
    model2 = MultiHeadAttention(num_heads=8, key_dim=64)(model2, model2)
    model2 = Dropout(0.4)(model2)
    model2 = LSTM(256, return_sequences=True)(model2)
    model2 = Dropout(0.4)(model2)
    model2 = LSTM(128, return_sequences=True)(model2)
    model2 = Dropout(0.4)(model2)
    model2 = GlobalAveragePooling1D()(model2)
    model2_output = Dense(64, activation='relu')(model2)
    model2_output = Dense(1)(model2_output)

    # Concatenate models
    ensemble = concatenate([model1_output, model2_output])
    merged_model = Dense(64, activation='relu')(ensemble)
    merged_model = Dense(1)(merged_model)

    # Build and compile model
    model = Model(inputs=[model1_input, model2_input], outputs=merged_model)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

    return model

# Compile and Train the Model
model = build_advanced_model()
model.summary()

# Callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, mode='min', min_delta=0.001),
    tf.keras.callbacks.ModelCheckpoint('./advanced_model.h5.keras', monitor='val_loss', save_best_only=True, mode='min')
]

# Train model
history = model.fit([x_train1, x_train2], y_train, epochs=500, batch_size=64,
                    validation_split=0.3, callbacks=callbacks)

# Load the best model weights
model.load_weights('./advanced_model.h5.keras')

# Predictions
train_pred = model.predict([x_train1, x_train2])
test_pred = model.predict([x_valid1, x_valid2])

# Evaluate and visualize results
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend()
plt.show()

# Plot predictions vs actuals
plt.scatter(y_train, train_pred)
plt.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'k--', lw=4)
plt.xlabel('Observed')
plt.ylabel('Predicted')
plt.show()

plt.scatter(y_valid, valid_pred)
plt.plot([y_valid.min(), y_valid.max()], [y_valid.min(), y_valid.max()], 'k--', lw=4)
plt.xlabel('Observed')
plt.ylabel('Predicted')
plt.show()


In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

# Calculate RMSE
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Calculate NRMSE
def nrmse(y_true, y_pred):
    return rmse(y_true, y_pred) / (np.max(y_true) - np.min(y_true))

# Calculate R-squared
def r_squared(y_true, y_pred):
    return r2_score(y_true, y_pred)

# Compute metrics on training and validation data
train_rmse = rmse(y_train, train_pred)
valid_rmse = rmse(y_valid, test_pred)

train_nrmse = nrmse(y_train, train_pred)
valid_nrmse = nrmse(y_valid, test_pred)

train_r2 = r_squared(y_train, train_pred)
valid_r2 = r_squared(y_valid, test_pred)

print(f"Training RMSE: {train_rmse:.4f}")
print(f"Validation RMSE: {test_rmse:.4f}")
print(f"Training NRMSE: {train_nrmse:.4f}")
print(f"Validation NRMSE: {test_nrmse:.4f}")
print(f"Training R-squared: {train_r2:.4f}")
print(f"Validation R-squared: {test_r2:.4f}")
